<a href="https://colab.research.google.com/github/cinnData/DataSci/blob/main/Notebooks/retail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example - RFM based segmentation in online retail transaction data

### Importing the data

In [ ]:
import pandas as pd

In [ ]:
path = 'https://raw.githubusercontent.com/cinnData/DataSci/main/Data/'
filename = path + 'retail.csv.zip'
df = pd.read_csv(filename)

In [ ]:
df.head()

In [ ]:
pd.crosstab(df['InvoiceNo'].str.contains('C'), df['Quantity'] < 0)

### Q1. New column with the number of days since the invoice was generated

In [ ]:
df['InvoiceDate'] = df['InvoiceDate'].astype('datetime64[D]')

In [ ]:
max_date = max(df['InvoiceDate'])
max_date

In [ ]:
df['Diff'] = (max_date - df['InvoiceDate']).dt.days
df.head()

### Q2a. Creating recency and frequency data

In [ ]:
RF = df.groupby('CustomerID')['Diff'].agg(['min', 'count'])
RF.head()

In [ ]:
RF.columns = ['Recency', 'Frequency']

### Q2b. Creating monetary data

In [ ]:
df['Monetary'] = df['Quantity']*df['UnitPrice']

In [ ]:
M = df.groupby('CustomerID')['Monetary'].sum()
M.head()

### Q2c. Joining the two data sets

In [ ]:
RFM = RF.merge(M, left_index=True, right_index=True)
RFM.head()

In [ ]:
RFM.describe()

### Q3b. Normalization

In [ ]:
import numpy as np

In [ ]:
def normalize(x): return (x - np.min(x))/(np.max(x) - np.min(x))

In [ ]:
RFM1 = RFM.apply(normalize, axis=0)

### Q3c. 8-cluster analysis

In [ ]:
import scipy.cluster.vq as cluster

In [ ]:
center = cluster.kmeans(RFM1, 8)[0]
center

In [ ]:
label = cluster.vq(RFM1, center)[0]
RFM['Segment'] = label
RFM.head()

In [ ]:
RFM['Segment'].value_counts()

### Q4a. Binarization

In [ ]:
RFM['BinRecency'] = ((RFM['Recency'] > RFM['Recency'].median()) + 0).astype(str)
RFM['BinFrequency'] = ((RFM['Frequency'] > RFM['Frequency'].median()) + 0).astype(str)
RFM['BinMonetary'] = ((RFM['Monetary'] > RFM['Monetary'].median()) + 0).astype(str)
RFM.head()

### Q4b. Compare this partition with that of the preceding question

In [ ]:
RFM['BinSegment'] = RFM['BinRecency'] + RFM['BinFrequency'] + RFM['BinMonetary']
RFM.head()

In [ ]:
pd.crosstab(RFM['Segment'], RFM['BinSegment'])